In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
headers = {'user-agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'}

In [ ]:
data_url=[]
data_title=[]
keywords=['颱風','天氣','歉收']#補上

In [ ]:
def find_text_ebc(items):
    data_text=[]
    items_text=items.find('div',class_="raw-style").find_all('p')
    for i in items_text:
        if '★' in i.text or '▼' in i.text or '圖／' in i.text or '' == i.text or '\xa0'==i.text or '▲' in i.text:
            continue
        else:
            if '●以上言論不代表東森' in i.text:
                break
            data_text.append(i.text.replace(u'\n',u'').replace(u'\xa0',u'').replace(u'\u3000',u'').strip())
    if []==data_text:
        text=soup.find('div',class_="raw-style").text.replace(u'\n',u'').replace(u'\xa0',u'')
        if '【往下' in text:
            text=text.split('【往下')[0]
            data_text.append(text)
    return str(data_text).replace('[','').replace(']','').replace("','",'').replace("'",'')
    
def find_reporter_ebc(items):
    reporter=items.find('span',class_="small-gray-text")
    reporter=reporter.text.split()[-1]
    if ':' in reporter:
        reporter='None'
    return reporter
def find_report_time_ebc(items):
    time=items.find('span',class_="small-gray-text")
    time=time.text.split()[0]
    return time

In [ ]:
for k in keywords:
    url_origin='https://news.ebc.net.tw/Search/Result?type=keyword&value='+k+'&page='
    for i in range(1,2):
        try:
            url=url_origin+str(i)
            print(url)
            resp=requests.get(url,headers=headers)
            soup=BeautifulSoup(resp.text,'html5lib')
            items=soup.find('div',class_="news-list-box")
            items_url=items.find_all('div',class_="style1 white-box")
            for i in items_url:
                data_url.append('https://news.ebc.net.tw'+i.a['href'])
                data_title.append(i.find("div",class_="title").text)
        except:
            print("No items to crawl")
            break
            print("---------------")



In [ ]:
if len(data_title)==len(data_url):
    data=pd.DataFrame()
    data['title']=data_title
    data['url']=data_url
    data['website']='東森新聞'
    print("make dataframe")
data.drop_duplicates(subset='url', keep='first', inplace=True)
data.reset_index(drop=True,inplace=True)

In [ ]:
data_reporter=[]
data_text=[]
data_time=[]


count=0
for i in data['url']:
    try:
        resp=requests.get(i,headers=headers)
        #time.sleep(1)
        items=BeautifulSoup(resp.text,"html5lib")
        #items=soup.find_all("div",class_="inner")
        try:
            x=find_reporter_ebc(items)
            data_reporter.append(x)
            print(x)
        except:
            data_reporter.append("None")
        try:
            y=find_text_ebc(items)
            data_text.append(y)
            print(y)
        except:
            data_text.append("None")
        try:
            z=find_report_time_ebc(items)
            data_time.append(z)
            print(z)
        except:
        
            data_time.append("None")
    except:
        print("fail,全部為None")
        data_text.append("None")
        data_time.append("None")
        data_reporter.append("None")
    count+=1
    print("目前執行次數 %d" % count)
    print("目前共有幾筆文章資料 %d" % len(data_text))
    print("目前共有幾筆時間資料 %d" % len(data_time))

In [ ]:
data['reporter']=data_reporter
data['time']=data_time
data['text']=data_text